In [5]:
# !pip install transformers
# !pip install datasets
# !pip install evaluate
!pip install "tf-models-official==2.13.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 20.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=742d4bd4117cf79dbeba76adf3f97a478249f77044cbeaf6611b4d69293a237e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [12]:
import torch
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import Dataset
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
from official.nlp import optimization

df=pd.read_csv("train.txt",sep="\t",names=["LABEL","REVIEW"])
dataset = Dataset.from_pandas(df)
dataset=dataset.rename_columns({"LABEL":"label","REVIEW":"text"})
dataset=dataset.train_test_split(test_size=0.2,shuffle=True)

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
def label_mapping(example):
    mapping={
        "DECEPTIVENEGATIVE":0,
        "DECEPTIVEPOSITIVE":1,
        "TRUTHFULNEGATIVE":2,
        "TRUTHFULPOSITIVE":3,
    }
    return {"label":mapping[example["label"]]}
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset=dataset.map(label_mapping,batched=False)
print(dataset["train"][0])
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
print(tokenized_datasets["train"][0])
tokenized_datasets.set_format("torch")

#
# tokenized_datasets["train"] = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
# tokenized_datasets["test"] = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

{'label': 2, 'text': "Beautiful historic hotel -- and since I'm in historic building business, was looking forward to stay. Moved rooms on second night because of loud party in adjacent suite, they escorted me to a room that was completely torn up -- duct work torn apart in bathroom, lamp fixtures literally falling off wall, mirror torn off wall, etc. Ended up in a room on 20th floor -- much nicer, but yesterday waited 10 minutes for elevator on two separate occasions. Only two of three elevators to top floors work -- and these two elevators skip floors, so you can be stuck waiting for ever. When I raised this to hotel staff, was told this was an old hotel and basically to stuff it. Umm, no. This is not an old building problem, this is a fixable elevator problem if anyone cared enough to fix it. Thumbs way down, stay away. Go to the Drake or Knickerbocker if you're looking for old world charm, sans hassle."}


Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

{'labels': 2, 'input_ids': [101, 3376, 3181, 3309, 1011, 1011, 1998, 2144, 1045, 1005, 1049, 1999, 3181, 2311, 2449, 1010, 2001, 2559, 2830, 2000, 2994, 1012, 2333, 4734, 2006, 2117, 2305, 2138, 1997, 5189, 2283, 1999, 5516, 7621, 1010, 2027, 13127, 2033, 2000, 1037, 2282, 2008, 2001, 3294, 7950, 2039, 1011, 1011, 23245, 2147, 7950, 4237, 1999, 5723, 1010, 10437, 17407, 6719, 4634, 2125, 2813, 1010, 5259, 7950, 2125, 2813, 1010, 4385, 1012, 3092, 2039, 1999, 1037, 2282, 2006, 3983, 2723, 1011, 1011, 2172, 3835, 2099, 1010, 2021, 7483, 4741, 2184, 2781, 2005, 7764, 2006, 2048, 3584, 6642, 1012, 2069, 2048, 1997, 2093, 19406, 2000, 2327, 8158, 2147, 1011, 1011, 1998, 2122, 2048, 19406, 13558, 8158, 1010, 2061, 2017, 2064, 2022, 5881, 3403, 2005, 2412, 1012, 2043, 1045, 2992, 2023, 2000, 3309, 3095, 1010, 2001, 2409, 2023, 2001, 2019, 2214, 3309, 1998, 10468, 2000, 4933, 2009, 1012, 26114, 1010, 2053, 1012, 2023, 2003, 2025, 2019, 2214, 2311, 3291, 1010, 2023, 2003, 1037, 8081, 3085, 7764

In [13]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=4)
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")
batch_size=8
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)
# steps_per_epoch = tf.data.experimental.cardinality(tf_train_dataset).numpy()
# num_train_steps = steps_per_epoch * epochs
# num_warmup_steps = int(0.1*num_train_steps)

# init_lr = 3e-5
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

H=model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
140/140 [==============================] - 4935s 35s/step - loss: 1.0996 - sparse_categorical_accuracy: 0.5536 - val_loss: 0.7154 - val_sparse_categorical_accuracy: 0.6857
Epoch 2/3
140/140 [==============================] - 4811s 34s/step - loss: 0.5378 - sparse_categorical_accuracy: 0.8313 - val_loss: 0.4480 - val_sparse_categorical_accuracy: 0.8107
Epoch 3/3
140/140 [==============================] - 4813s 34s/step - loss: 0.2957 - sparse_categorical_accuracy: 0.9116 - val_loss: 0.3642 - val_sparse_categorical_accuracy: 0.8500
